Meta-learner - a prediction model combining statstical and machine learning models      
For simplicity, we use ridge regression as the statistical model. The machine learning model and meta-learner are based on the SVM architecture.

In [1]:
import numpy as np
import sklearn as sk
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# generate a toy 2d classification dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# split to base:meta:test = 400:400:200
X_base, X_temp, y_base, y_temp = train_test_split(X, y, test_size=0.6, random_state=1)
X_meta, X_test, y_meta, y_test = train_test_split(X_temp, y_temp, test_size=0.333, random_state=1)
print('X_shape = {}, y_shape = {}'.format(np.shape(X), np.shape(y)))
print('N_predictor = {}, N_class = {}'.format(np.shape(X)[1], len(np.unique(y))))

X_shape = (1000, 20), y_shape = (1000,)
N_predictor = 20, N_class = 2


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# statistical model: logistic regression with L2 regularization
stat = LogisticRegression().fit(X_base, y_base)
# get probability
pred_meta_stat, pred_test_stat = stat.predict_proba(X_meta)[:, 1], stat.predict_proba(X_test)[:, 1]

# machine learning model: SVM
ml = SVC(probability=True).fit(X_base, y_base)
pred_meta_ml, pred_test_ml = ml.predict_proba(X_meta)[:, 1], ml.predict_proba(X_test)[:, 1]

# meta-learner: SVM
meta = SVC(probability=True).fit(np.c_[X_meta, pred_meta_stat, pred_meta_ml], y_meta)

In [3]:
# AUC evaluation
from sklearn import metrics
def my_auc(y, pred):
    fp, tp, _ = metrics.roc_curve(y, pred)
    return metrics.auc(fp, tp)

print('AUC_stat = {:0.4f} \nAUC_ml = {:0.4f} \nAUC_meta = {:0.4f}'.format(
            my_auc(y_test, stat.predict_proba(X_test)[:, 1]),
            my_auc(y_test, ml.predict_proba(X_test)[:, 1]), 
            my_auc(y_test, meta.predict_proba(np.c_[X_test, pred_test_stat, pred_test_ml])[:, 1])))

AUC_stat = 0.9475 
AUC_ml = 0.9772 
AUC_meta = 0.9800
